# 20 vs 20 signal processing

### There is 2 ways to process signal
* Based on exact tags count using fragment size
    UNIQUE_BAM -> PILEUP_BED -> TAGS -> intersect with given regions bed and compute intersection
* Based on bigwigs
    UNIQUE_BAM -> BIGWIG -> bigWigAverageOverBed

### Interesting LOCI
/mnt/stripe/bio/raw-data/aging/loci_of_interest/

### Prepare BigWigs
```
mkdir /mnt/stripe/bio/experiments/signal
cd /mnt/stripe/bio/experiments/signal
mkdir H3K27ac
cp /mnt/stripe/bio/experiments/configs/Y20O20/browsers/browser_tracks/*k27ac*_unique.bw H3K27ac
mkdir H3K27me3
cp /mnt/stripe/bio/experiments/configs/Y20O20/browsers/browser_tracks/*k27me3*_unique.bw H3K27me3
mkdir H3K36me3
cp /mnt/stripe/bio/experiments/configs/Y20O20/browsers/browser_tracks/*k36me3*_unique.bw H3K36me3
mkdir H3K4me1
cp /mnt/stripe/bio/experiments/configs/Y20O20/browsers/browser_tracks/*k4me1*_unique.bw H3K4me1/
mkdir H3K4me3
cp /mnt/stripe/bio/experiments/configs/Y20O20/browsers/browser_tracks/*k4me3*_unique.bw H3K4me3/
mkdir meth
cp /mnt/stripe/bio/experiments/configs/Y20O20/browsers/browser_tracks/*mlevel*.bw meth/
```

### Process signals and build PCA/graphics/diffbind scores
```
export PYTHONPATH="/mnt/stripe/washu:$PYTHONPATH"
DIR=/mnt/stripe/bio/experiments/signal
for M in $(ls $DIR | grep -v log.txt); do 
    echo "Processing $DIR/$M"; 
    for F in $(find /mnt/stripe/bio/raw-data/aging/loci_of_interest/ -name "*.bed"); do 
        echo "Processing regions $F"; 
        N=${F%%.bed}; 
        N=${N##*/}; 
        echo $N; 
        bash /mnt/stripe/washu/parallel/signals_bw.sh $DIR/$M $F $N /mnt/stripe/bio/genomes/hg19/hg19.chrom.sizes;         done;
done | tee log.txt
```

### Copy pngs to a single folders
```
# Copy results to _png
for M in $(ls $DIR | grep -v log.txt); do 
    mkdir -p "${M}_png";
    find $M/ -name "*.png" | xargs -i cp {} ${M}_png/
done
```

In [ ]:
%matplotlib inline
%run ~/work/washu/scripts/signals_visualize.py

# K27ac

In [ ]:
folder = '/mnt/stripe/bio/experiments/signal/H3K27ac'
for f in os.listdir(folder):
    if (os.path.isdir(os.path.join(folder, f))):
        process(folder, f)

# K27me3

In [ ]:
folder = '/mnt/stripe/bio/experiments/signal/H3K27me3'
for f in os.listdir(folder):
    if (os.path.isdir(os.path.join(folder, f))):
        process(folder, f)

# K36me3

In [ ]:
folder = '/mnt/stripe/bio/experiments/signal/H3K36me3'
for f in os.listdir(folder):
    if (os.path.isdir(os.path.join(folder, f))):
        process(folder, f)

# K4me1

In [ ]:
folder = '/mnt/stripe/bio/experiments/signal/H3K4me1'
for f in os.listdir(folder):
    if (os.path.isdir(os.path.join(folder, f))):
        process(folder, f)

# K4me3

In [ ]:
folder = '/mnt/stripe/bio/experiments/signal/H3K4me3'
for f in os.listdir(folder):
    if (os.path.isdir(os.path.join(folder, f))):
        process(folder, f)